In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

sys.path.append("..")

In [3]:
import string

In [4]:
import settings
from funcs.utils import find_project_root

ROOT = find_project_root()

In [5]:
import pandas as pd

In [6]:
from transformers import BertTokenizerFast

get candidate tokens from the poetry token

In [7]:
source_path = ROOT / settings.path_to_chinese_poetry_corpus
disallowed_words = ["："] + list(string.ascii_letters) + list(string.digits)
disallow_words = disallowed_words + settings.disallowed_words

In [8]:
with source_path.open() as f:
    docs = f.readlines()

In [9]:
words = list(set([_ for doc in docs for _ in doc if _ not in disallowed_words]))

In [10]:
words[:5]

['砺', '盂', '均', '禊', '模']

In [11]:
tokenizer = BertTokenizerFast.from_pretrained(settings.chinese_bert_model_name)
tokenizer

In [12]:
encodings = tokenizer(words, add_special_tokens=False)
input_ids = encodings["input_ids"]

In [13]:
assert len(input_ids) == len(words)

In [14]:
example_ids = [_ for item in input_ids[:5] for _ in item]
example_ids

[4791, 4655, 1772, 100, 3563]

In [15]:
tokenizer.convert_ids_to_tokens(example_ids)

['砺', '盂', '均', '[UNK]', '模']

In [16]:
flat_input_ids = [_ for item in input_ids for _ in item]
tokens = tokenizer.convert_ids_to_tokens(flat_input_ids)

In [17]:
poetry_df = pd.DataFrame({"token": tokens, "id": flat_input_ids})
poetry_df

,token,id
0,砺,4791
1,盂,4655
2,均,1772
3,[UNK],100
4,模,3563
...,...,...
7617,[UNK],100
7618,[UNK],100
7619,惶,2684
7620,逢,6864


In [18]:
poetry_df = poetry_df[poetry_df["token"] != "[UNK]"]
poetry_df

,token,id
0,砺,4791
1,盂,4655
2,均,1772
4,模,3563
5,燔,4241
...,...,...
7615,驮,7718
7616,泠,3795
7619,惶,2684
7620,逢,6864


In [20]:
path_to_vocab = ROOT / "datasets" / "output" / "poetry_vocab.csv"
poetry_df.to_csv(path_to_vocab, index=False)